## 上传文件

In [31]:
from openai import OpenAI

client = OpenAI(
    api_key="hidden api key"
)

uploaded_file = client.files.create(
  file=open("hjz_fine_tuning_dataset.jsonl", "rb"),
  purpose="fine-tune"
)

print(uploaded_file)

FileObject(id='file-1PoMg6Khx2fe1sc1dBDziL', bytes=52157, created_at=1740557147, filename='hjz_fine_tuning_dataset.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None)


## 创建微调任务

In [33]:
client.fine_tuning.jobs.create(
    training_file="file-1PoMg6Khx2fe1sc1dBDziL", model="gpt-4o-2024-08-06"
)

FineTuningJob(id='ftjob-Th8WFWnZWF38fjTHyMcfsKe1', created_at=1740557198, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-zVEUtkATt40mFSAeRowopASm', result_files=[], seed=1648897910, status='validating_files', trained_tokens=None, training_file='file-1PoMg6Khx2fe1sc1dBDziL', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix=None)

## 打印微调任务状态

In [76]:
events = client.fine_tuning.jobs.list_events(
  fine_tuning_job_id="ftjob-Th8WFWnZWF38fjTHyMcfsKe1",
)

for data in events.data:
    print(data.message)

The job has successfully completed
New fine-tuned model created
Checkpoint created at step 106
Checkpoint created at step 53
Step 159/159: training loss=0.69
Step 158/159: training loss=0.53
Step 157/159: training loss=0.27
Step 156/159: training loss=0.11
Step 155/159: training loss=0.67
Step 154/159: training loss=0.45
Step 153/159: training loss=0.39
Step 152/159: training loss=0.31
Step 151/159: training loss=0.35
Step 150/159: training loss=0.18
Step 149/159: training loss=0.55
Step 148/159: training loss=0.49
Step 147/159: training loss=0.13
Step 146/159: training loss=0.46
Step 145/159: training loss=0.22
Step 144/159: training loss=0.38


## 获取最新一个模型名称

In [77]:
job = client.fine_tuning.jobs.list(limit=10)
print(job)


SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-Th8WFWnZWF38fjTHyMcfsKe1', created_at=1740557198, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-2024-08-06:personal::B56tSE3Q', finished_at=1740557836, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=3), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-zVEUtkATt40mFSAeRowopASm', result_files=['file-9ZCq7phrvQCa41iLZB2D7U'], seed=1648897910, status='succeeded', trained_tokens=22740, training_file='file-1PoMg6Khx2fe1sc1dBDziL', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=3)), type='supervised'), user_provided_suffix=None), FineTuningJob(id='ftjob-8DcbeViQJMBOE4JVXW2lSxkK', created_at=1740538785, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt

## 对话demo

### 执行这个单元格开始一轮新对话

In [96]:
conversation = [
    {"role": "system", "content": "You are a helpful assistant. You need to instruct the user on how to operate the Android phone step by step."}
]


### 每轮对话中修改user_input, 然后执行该单元格

In [103]:
user_input = "I have uninstalled the unused app, thank you !" 
conversation.append({"role": "user", "content": user_input})

completion = client.chat.completions.create(
    model="ft:gpt-4o-2024-08-06:personal::B56tSE3Q",
    messages=conversation
)
assistant_reply = completion.choices[0].message

conversation.append({"role": "assistant", "content": assistant_reply.content})

for conversation_item in conversation:
    print(conversation_item["role"] + ": " + conversation_item["content"])

system: You are a helpful assistant. You need to instruct the user on how to operate the Android phone step by step.
user: I want to free some space on my phone
assistant: First, open the Settings app on your Android device.
user: I have opened it, what's next?
assistant: Scroll down and tap on 'Storage'.
user: I did not found storage, what should I do?
assistant: It might be labeled as 'Device Care' or 'Device Maintenance' on some phones.
user: I saw device maintance and entered it. What should I do next?
assistant: Now, tap on 'Storage' to view the breakdown of your used and available space.
user: I can see the detail of my space. How can I free up space?
assistant: Tap on 'Clean Now' or select unused apps and files to delete them.
user: I cannot see clean now button, but there is a 'unused apps' button
assistant: Great! Tap on 'Unused Apps' to review and delete any apps you no longer need.
user: I have uninstalled the unused app, thank you !
assistant: You're welcome! Let me know if